In [1]:
import sys
sys.path.insert(0,'./caffe-fast-rcnn/python/')
import caffe
import matplotlib.pyplot as plt 
%matplotlib inline
import numpy as np
sys.path.insert(0,'lib/')
sys.path.insert(0,'tools/')

In [2]:
from fast_rcnn.config import cfg, cfg_from_file, cfg_from_list
from fast_rcnn.bbox_transform import clip_boxes, bbox_transform_inv
from fast_rcnn.nms_wrapper import nms
import caffe
import time, os, sys
import cv2
import numpy as np
import _init_paths
from matplotlib.patches import Polygon
import scipy
from utils.for_demo import im_proccess, vis_detections, colormap

caffe.set_mode_gpu()
caffe.set_device(0)
test_prototxt = 'models/coco/ResNet-50/mask_rcnn_end2end/maskrcnn_resnet50_C4_test.prototxt'
weights = 'output/faster_rcnn_end2end/coco_2014_train/coco_train_mask_rcnn_iter_300000.caffemodel'
net = caffe.Net(test_prototxt, weights, caffe.TEST)


In [3]:
max_per_image = 100
num_classes = 81
score_thresh = 0.05
mask_thresh = 0.4
vis = 0
mask_color_id = 0
def get_boxes_masks(im_orig, rois, im_scale, out, i):
    boxes = rois[:, 1:5] / im_scale
    scores = out['cls_prob']
    box_deltas = out['bbox_pred']
    masks = np.squeeze(out['mask_prob'])
    pred_boxes = bbox_transform_inv(boxes, box_deltas)
    pred_boxes = clip_boxes(pred_boxes, im_orig.shape)

    # skip j = 0, because it's the background class
    for j in xrange(1, num_classes):
        inds = np.where(scores[:, j] > score_thresh)[0]
        cls_scores = scores[inds, j]
        cls_boxes = pred_boxes[inds, j*4:(j+1)*4]
        cls_masks = masks[inds, ...]
        cls_dets = np.hstack((cls_boxes, cls_scores[:, np.newaxis])).astype(np.float32, copy=False)
        keep = nms(cls_dets, cfg.TEST.NMS)
        cls_dets = cls_dets[keep, :]
    #     print np.argsort(cls_dets[:, -1])
    #     print j,keep
        cls_masks = cls_masks[keep, :]
        if vis:
            vis_detections(im_orig, j, cls_dets)
        all_data[j][i] = {'dets':cls_dets, 'masks': cls_masks}

    # Limit to max_per_image detections *over all classes*
    if max_per_image > 0:
        image_scores = np.hstack([all_data[j][i]['dets'][:, -1] for j in xrange(1, num_classes)])
        if len(image_scores) > max_per_image:
            image_thresh = np.sort(image_scores)[-max_per_image]
            for j in xrange(1, num_classes):
                keep = np.where(all_data[j][i]['dets'][:, -1] >= image_thresh)[0]
                all_data[j][i] = {'dets':all_data[j][i]['dets'][keep, :], 'masks': all_data[j][i]['masks'][keep, :]}

In [4]:
im_dir = 'data/test_images/'
save_dir = 'data/save/'
classes = np.load('classes.npy')
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
im_list = os.listdir(im_dir)
num_images = len(im_list)
color_list = colormap(rgb=True) / 255   
all_data = [[[] for _ in xrange(num_images)] for _ in xrange(num_classes)]
for i in range(num_images):
    im_orig = cv2.imread(im_dir + im_list[i])
    image_shape=im_orig.shape 
    im_blob, im_info, im_scale = im_proccess(im_orig)
    net.blobs['data'].reshape(*(im_blob.shape))
    net.blobs['im_info'].reshape(*(im_info.shape))
    net.blobs['data'].data[...] = im_blob
    net.blobs['im_info'].data[...] = im_info
    out = net.forward()
    rois = net.blobs['rois'].data.copy()
    get_boxes_masks(im_orig, rois, im_scale, out, i)
    
    
    fig = plt.gca()
    fig.imshow(im_orig[:,:,::-1])
    for j in xrange(1, num_classes):
        color_mask = color_list[mask_color_id % len(color_list), 0:3]
        mask_color_id += 1
        w_ratio = .4
        for c in range(3):
            color_mask[c] = color_mask[c] * (1 - w_ratio) + w_ratio
        if len(all_data[j][i]['dets']) > 0:
            mask = all_data[j][i]['masks'][0, ...]
            bbox = all_data[j][i]['dets'][0, :4]

            x1, y1, x2, y2 = bbox
            mask = scipy.misc.imresize(mask, (y2 - y1, x2 - x1), interp='bilinear')
            _, mask=cv2.threshold(mask, mask_thresh*255, 255, cv2.THRESH_BINARY)

            full_mask = np.zeros(image_shape[:2], dtype=np.uint8)
            full_mask[int(round(y1)):(mask.shape[0]+int(round(y1))), int(round(x1)):(mask.shape[1]+int(round(x1)))] = mask

            fig.add_patch(plt.Rectangle((bbox[0], bbox[1]),
                                               bbox[2] - bbox[0],
                                               bbox[3] - bbox[1], fill=False,
                                               edgecolor='g', linewidth=1.2))
            
            class_text = classes[j] + ' {:0.2f}'.format(all_data[j][i]['dets'][0, -1]).lstrip('0')
            fig.text(bbox[0], bbox[1] - 2,
                           class_text,
                           fontsize = 4,
                           family = 'serif',
                           bbox = dict(facecolor='g', alpha=0.4, pad=0, edgecolor='none'),
                           color = 'white')

            _, contour, _ = cv2.findContours(
                full_mask.copy(), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)

            for c in contour:
                polygon = Polygon(
                    c.reshape((-1, 2)),
                    fill=True, facecolor=color_mask,
                    edgecolor='w', linewidth=0.7,
                    alpha=0.5)
                fig.add_patch(polygon)
    plt.savefig(save_dir+im_list[i], dpi=200)
    plt.clf()